In [4]:
import cv2
import numpy as np
from tkinter import *
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
def open_image():
    global img, img_path, tk_img

    img_path = filedialog.askopenfilename(
        filetypes=[("Image Files", "*.jpg *.png *.jpeg")]
    )
    if img_path:
        img = cv2.imread(img_path)
        display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        display = Image.fromarray(display)
        display = display.resize((300, 300))
        tk_img = ImageTk.PhotoImage(display)
        canvas.create_image(0, 0, anchor=NW, image=tk_img)


def count_red_pixels():
    if img is None:
        messagebox.showerror("Error", "No image loaded!")
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    R, G, B = img_rgb[:,:,0], img_rgb[:,:,1], img_rgb[:,:,2]

    red_pixels = np.where((R > 150) & (G < 100) & (B < 100))
    count = len(red_pixels[0])

    messagebox.showinfo("Red Pixels", f"Number of red pixels: {count}")


def convert_grayscale():
    if img is None:
        messagebox.showerror("Error", "No image loaded!")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("grayscale_output.jpg", gray)
    messagebox.showinfo("Saved", "Grayscale image saved as grayscale_output.jpg")


def dominant_color():
    if img is None:
        messagebox.showerror("Error", "No image loaded!")
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pixels = img_rgb.reshape((-1, 3)).astype(np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.2)
    _, labels, centers = cv2.kmeans(pixels, 3, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    counts = np.bincount(labels.flatten())
    dominant = centers[np.argmax(counts)]

    messagebox.showinfo("Dominant Color", f"Dominant Color (R,G,B): {tuple(dominant)}")


# ----------------- GUI Setup -----------------
root = Tk()
root.title("Mini Image Processing Tool")
root.geometry("600x400")

img = None
img_path = ""

canvas = Canvas(root, width=300, height=300, bg="lightgray")
canvas.pack(side=LEFT, padx=10, pady=10)

btn_frame = Frame(root)
btn_frame.pack(side=RIGHT, padx=20)

Button(btn_frame, text="Open Image", width=20, command=open_image).pack(pady=10)
Button(btn_frame, text="Count Red Pixels", width=20, command=count_red_pixels).pack(pady=10)
Button(btn_frame, text="Convert to Grayscale", width=20, command=convert_grayscale).pack(pady=10)
Button(btn_frame, text="Detect Dominant Color", width=20, command=dominant_color).pack(pady=10)

root.mainloop()
